In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets
import cv2

# GPU 사용 가능하면 GPU로 모델 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# 학습 함수
def train(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# 평가 함수
def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            _, predictions = torch.max(outputs, 1)

            running_loss += loss.item() * images.size(0)
            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    accuracy = correct_predictions / total_predictions
    return epoch_loss, accuracy

In [3]:
# 이미지 추가 전처리
def apply_gaussian_filter(image):
    np_image = image.numpy()  # 텐서를 넘파이 배열로 변환
    if len(np_image.shape) == 3:
        np_image = np_image.transpose(1, 2, 0)  # 차원 변경 (C, H, W) -> (H, W, C)
        filtered_image = cv2.GaussianBlur(np_image, (3, 3), 0)
        filtered_image = filtered_image.transpose(2, 0, 1)  # 차원 변경 (H, W, C) -> (C, H, W)
        return torch.from_numpy(filtered_image)  # 넘파이 배열을 텐서로 변환
    else:
        return image

In [4]:
# 데이터 로딩 및 전처리
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomRotation(degrees=30),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(size=224),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: apply_gaussian_filter(x)), # 가우시안 필터링 적용
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 데이터셋 로드
train_dataset = datasets.ImageFolder(root='../data/Multi_classification/train', transform=transform)
test_dataset = datasets.ImageFolder(root='../data/Multi_classification/test', transform=transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# 모델 정확도가 일정 수준이 될 때까지
test_accuracy = 0
while test_accuracy < 0.54:
    
    # 모델 정의
    model = models.shufflenet_v2_x1_0(pretrained=True)
    num_classes = 9  # 분류할 클래스 수

    # 마지막 레이어 수정 (다중 분류를 위한 출력 노드 수 변경)
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    # 손실 함수와 옵티마이저 정의
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    model.to(device)

    # 학습 및 평가
    num_epochs = 30
    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer)
        test_loss, test_accuracy = evaluate(model, test_loader, criterion)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

        if test_accuracy >= 0.54:
            break

C:\Users\tjoeun\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\tjoeun\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/30], Train Loss: 1.8017, Test Loss: 1.7232, Test Accuracy: 0.3711
Epoch [2/30], Train Loss: 1.5290, Test Loss: 1.7418, Test Accuracy: 0.3867
Epoch [3/30], Train Loss: 1.4125, Test Loss: 1.6201, Test Accuracy: 0.4200
Epoch [4/30], Train Loss: 1.3380, Test Loss: 1.5501, Test Accuracy: 0.4689
Epoch [5/30], Train Loss: 1.2858, Test Loss: 1.4861, Test Accuracy: 0.4844
Epoch [6/30], Train Loss: 1.2574, Test Loss: 1.4874, Test Accuracy: 0.4689
Epoch [7/30], Train Loss: 1.2086, Test Loss: 1.4238, Test Accuracy: 0.5067
Epoch [8/30], Train Loss: 1.1893, Test Loss: 1.4325, Test Accuracy: 0.5200
Epoch [9/30], Train Loss: 1.1363, Test Loss: 1.4902, Test Accuracy: 0.4533
Epoch [10/30], Train Loss: 1.1325, Test Loss: 1.5177, Test Accuracy: 0.4556
Epoch [11/30], Train Loss: 1.1003, Test Loss: 1.6394, Test Accuracy: 0.4756
Epoch [12/30], Train Loss: 1.0764, Test Loss: 1.6476, Test Accuracy: 0.4889
Epoch [13/30], Train Loss: 1.0453, Test Loss: 1.5594, Test Accuracy: 0.4844
Epoch [14/30], Train 

In [6]:
import torch
from sklearn.metrics import confusion_matrix

def calculate_confusion_matrix(model, dataloader, device):
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    confusion_mat = confusion_matrix(all_labels, all_predictions)
    return confusion_mat

In [7]:
calculate_confusion_matrix(model, test_loader, device)

array([[15,  4,  8,  8,  6,  3,  1,  5,  0],
       [ 7, 33,  1,  3,  0,  5,  0,  1,  0],
       [ 5,  2, 28,  5,  0,  1,  2,  7,  0],
       [ 4,  3,  3, 26,  8,  4,  1,  0,  1],
       [ 2,  2,  2,  5, 15,  7,  7,  8,  2],
       [ 4,  2,  1,  5,  4, 20,  6,  3,  5],
       [ 3,  2,  2,  5,  3,  7, 25,  3,  0],
       [ 1,  3,  3,  2,  5,  7,  6, 22,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 50]], dtype=int64)

In [8]:
print(model.fc.weight[0][:10])
print(model.fc.bias[:10])

tensor([-0.1203, -0.1653, -0.1512,  0.0428, -0.1532, -0.0063, -0.0070, -0.1020,
        -0.0540, -0.0223], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([-6.4833e-03, -2.7859e-02,  2.7876e-02, -1.5922e-02,  3.5121e-02,
        -2.6620e-02,  2.9105e-03, -1.6452e-02, -2.3733e-05], device='cuda:0',
       grad_fn=<SliceBackward0>)


In [9]:
torch.save(model, 'MultiClassification.h5')